In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import asyncio
from aiohttp import ClientSession
from concurrent.futures import ThreadPoolExecutor

In [2]:
df=pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')
df.fillna(float("NaN"), inplace=True)

df.head()

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982067 entries, 0 to 1982066
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

In [ ]:
df.describe()


,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
count,1.755258e+06,1.755258e+06,1.982049e+06,1.982036e+06,1.982067e+06,1.982067e+06,1.982067e+06,1.982067e+06,1.982067e+06,1.982067e+06,1.982067e+06
mean,4.062805e+01,-7.375290e+01,2.974659e-01,1.420257e-03,5.469694e-02,7.209645e-04,2.552184e-02,1.074636e-04,2.145942e-01,5.736436e-04,3.088706e+06
std,1.978253e+00,3.725428e+00,6.884435e-01,3.969451e-02,2.402712e-01,2.734391e-02,1.595896e-01,1.041447e-02,6.496194e-01,2.608206e-02,1.503442e+06
min,0.000000e+00,-2.013600e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01
25%,4.066797e+01,-7.397495e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.131636e+06
50%,4.072117e+01,-7.392735e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.627242e+06
75%,4.076945e+01,-7.386661e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.123012e+06
max,4.334444e+01,0.000000e+00,4.300000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,4.300000e+01,5.000000e+00,4.618874e+06


In [ ]:
null_porcentage=(df.isnull().sum()/len(df))*100
null_porcentage

CRASH DATE                        0.000000
CRASH TIME                        0.000000
BOROUGH                          31.095064
ZIP CODE                         31.107072
LATITUDE                         11.443054
LONGITUDE                        11.443054
LOCATION                         11.443054
ON STREET NAME                   20.913773
CROSS STREET NAME                37.164788
OFF STREET NAME                  83.746564
NUMBER OF PERSONS INJURED         0.000908
NUMBER OF PERSONS KILLED          0.001564
NUMBER OF PEDESTRIANS INJURED     0.000000
NUMBER OF PEDESTRIANS KILLED      0.000000
NUMBER OF CYCLIST INJURED         0.000000
NUMBER OF CYCLIST KILLED          0.000000
NUMBER OF MOTORIST INJURED        0.000000
NUMBER OF MOTORIST KILLED         0.000000
CONTRIBUTING FACTOR VEHICLE 1     0.310333
CONTRIBUTING FACTOR VEHICLE 2    15.140154
CONTRIBUTING FACTOR VEHICLE 3    92.951298
CONTRIBUTING FACTOR VEHICLE 4    98.424069
CONTRIBUTING FACTOR VEHICLE 5    99.575847
COLLISION_I

# Data Cleaning

### Reduction of dimensions 

In [3]:
#Column CRASH DATE and CRASH TIME merge
df['CRASH_DATETIME']=pd.to_datetime(df['CRASH DATE']+ ' '+ df['CRASH TIME'])
df.drop(['CRASH DATE', 'CRASH TIME'], axis=1, inplace=True)


In [4]:
df.drop(['COLLISION_ID'],axis=1, inplace=True)

In [5]:
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])
df.drop(['CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5','VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4','VEHICLE TYPE CODE 5','OFF STREET NAME'],axis=1, inplace=True)
#df.drop(['COLLISION_ID'],axis=1, inplace=True)
df


#Sorted Columns
columns=['CRASH_DATETIME']+ [col for col in df.columns if col != 'CRASH_DATETIME']
df=df[columns]

df

,CRASH_DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
6,2021-12-14 17:05:00,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Sedan,Tractor Truck Diesel
7,2021-12-14 08:17:00,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,Sedan,Sedan
8,2021-12-14 21:10:00,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inexperience,Unspecified,Sedan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982061,2023-04-04 22:25:00,QUEENS,11377.0,40.749700,-73.902565,"(40.7497, -73.902565)",60 STREET,37 AVENUE,5.0,0.0,0,0,0,0,5,0,Driver Inattention/Distraction,Unspecified,Sedan,Sedan
1982062,2023-04-04 21:40:00,QUEENS,11356.0,40.790695,-73.845505,"(40.790695, -73.845505)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
1982064,2023-04-04 20:10:00,NaN,NaN,40.672382,-73.785740,"(40.672382, -73.78574)",BAISLEY BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan
1982065,2023-03-29 11:05:00,MANHATTAN,10011.0,40.735520,-73.993990,"(40.73552, -73.99399)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,E-Bike,Sedan


DF to chek if there is information that can be completed   

In [6]:
selected_columns1 = ['BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE']
new_df = df[selected_columns1]

# Eliminar las filas que tienen datos en ambas columnas 'ZIP CODE' y 'BOROUGH'
filtered_df = new_df.query('BOROUGH.isnull() or `ZIP CODE`.isnull()', engine='python')

#Take sample
sample_df = filtered_df.sample(frac=0.01)


print(sample_df)

        BOROUGH ZIP CODE   LATITUDE  LONGITUDE
1664463     NaN      NaN  40.691915 -73.883315
615859      NaN      NaN  40.756107 -73.900710
706953      NaN      NaN  40.571020 -74.115715
726702      NaN      NaN  40.757797 -73.899765
631401      NaN      NaN  40.823640 -73.879940
...         ...      ...        ...        ...
460753      NaN      NaN  40.802074 -73.934070
29508       NaN      NaN  40.836754 -73.875084
882895      NaN      NaN  40.836090 -73.870390
1413137     NaN      NaN  40.613521 -73.926783
896378      NaN      NaN  40.763588 -73.971420

[4255 rows x 4 columns]


The code checks a sample of the dataset, if all the sample in BORought and ZIP CODE after searching it with its coordinates is null, all rows with null municipality and postal code will be removed because the provided coordinates do not exist

In [ ]:
async def get_async(session, latitud, longitud):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitud}&lon={longitud}&zoom=18&addressdetails=1"
    async with session.get(url) as response:
        datos = await response.json()
        direccion = datos.get('address', {})
        return direccion

async def get_borought_zip_code_async(session, latitud, longitud):
    direccion = await get_async(session, latitud, longitud)
    barrio = direccion.get('borough', direccion.get('suburb'))
    codigo_postal = direccion.get('postcode')
    return barrio, codigo_postal

async def process_row_async(row):
    async with ClientSession() as session:
        return await get_borought_zip_code_async(session, row[0], row[1])

def process_row(row):
    return asyncio.run(process_row_async(row))




first_chunk = sample_df

#amount of threads
num_threads = 10

# Processing in paralell
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_row, first_chunk.itertuples(index=False)))

sample[['Borough', 'Postcode']] = pd.DataFrame(results, columns=['Borough', 'Postcode'])


print(sample)


              id BOROUGH ZIP CODE   LATITUDE  LONGITUDE Borough Postcode
324747    324747     NaN      NaN  40.744450 -73.973040     NaN      NaN
735224    735224     NaN      NaN  40.738194 -73.849200     NaN      NaN
1076522  1076522     NaN      NaN  40.770770 -73.917270     NaN      NaN
230928    230928     NaN      NaN  40.710926 -73.739960     NaN      NaN
1373319  1373319     NaN      NaN  40.738736 -73.814271     NaN      NaN
...          ...     ...      ...        ...        ...     ...      ...
201620    201620     NaN      NaN  40.830563 -73.927270     NaN      NaN
918774    918774     NaN      NaN  40.734070 -73.744790     NaN      NaN
1614981  1614981     NaN      NaN  40.870782 -73.819219     NaN      NaN
792346    792346     NaN      NaN  40.707153 -73.818900     NaN      NaN
623234    623234     NaN      NaN  40.820053 -73.926660     NaN      NaN

[4255 rows x 7 columns]


### Removing data null of BOROUGHT and ZIP_CODE

In [7]:
df_cleaned = df.dropna(subset=['BOROUGH', 'ZIP CODE'])
df.drop(['LOCATION'], axis=1, inplace=True)


(df_cleaned)

,CRASH_DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7,2021-12-14 08:17:00,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,Sedan,Sedan
8,2021-12-14 21:10:00,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inexperience,Unspecified,Sedan,NaN
9,2021-12-14 14:58:00,MANHATTAN,10017.0,40.751440,-73.973970,"(40.75144, -73.97397)",3 AVENUE,EAST 43 STREET,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982059,2023-04-04 14:58:00,BRONX,10465.0,40.819572,-73.820724,"(40.819572, -73.820724)",SAMPSON AVENUE,QUINCY AVENUE,2.0,0.0,0,0,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan
1982061,2023-04-04 22:25:00,QUEENS,11377.0,40.749700,-73.902565,"(40.7497, -73.902565)",60 STREET,37 AVENUE,5.0,0.0,0,0,0,0,5,0,Driver Inattention/Distraction,Unspecified,Sedan,Sedan
1982062,2023-04-04 21:40:00,QUEENS,11356.0,40.790695,-73.845505,"(40.790695, -73.845505)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
1982065,2023-03-29 11:05:00,MANHATTAN,10011.0,40.735520,-73.993990,"(40.73552, -73.99399)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,E-Bike,Sedan


# The data hass been cleaned, the percentaje of lost data is minimal

In [11]:
#LOST DATA

null_porcentage=(df_cleaned.isnull().sum()/len(df))*100
null_porcentage

CRASH_DATETIME                    0.000000
BOROUGH                           0.000000
ZIP CODE                          0.000000
LATITUDE                          0.000000
LONGITUDE                         0.000000
LOCATION                          0.000000
ON STREET NAME                   16.087550
CROSS STREET NAME                16.117232
NUMBER OF PERSONS INJURED         0.000627
NUMBER OF PERSONS KILLED          0.001310
NUMBER OF PEDESTRIANS INJURED     0.000000
NUMBER OF PEDESTRIANS KILLED      0.000000
NUMBER OF CYCLIST INJURED         0.000000
NUMBER OF CYCLIST KILLED          0.000000
NUMBER OF MOTORIST INJURED        0.000000
NUMBER OF MOTORIST KILLED         0.000000
CONTRIBUTING FACTOR VEHICLE 1     0.259164
CONTRIBUTING FACTOR VEHICLE 2    12.109559
VEHICLE TYPE CODE 1               0.488817
VEHICLE TYPE CODE 2              14.535185
dtype: float64

In [12]:
df_cleaned

,CRASH_DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7,2021-12-14 08:17:00,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,Sedan,Sedan
8,2021-12-14 21:10:00,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inexperience,Unspecified,Sedan,NaN
9,2021-12-14 14:58:00,MANHATTAN,10017.0,40.751440,-73.973970,"(40.75144, -73.97397)",3 AVENUE,EAST 43 STREET,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982059,2023-04-04 14:58:00,BRONX,10465.0,40.819572,-73.820724,"(40.819572, -73.820724)",SAMPSON AVENUE,QUINCY AVENUE,2.0,0.0,0,0,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan
1982061,2023-04-04 22:25:00,QUEENS,11377.0,40.749700,-73.902565,"(40.7497, -73.902565)",60 STREET,37 AVENUE,5.0,0.0,0,0,0,0,5,0,Driver Inattention/Distraction,Unspecified,Sedan,Sedan
1982062,2023-04-04 21:40:00,QUEENS,11356.0,40.790695,-73.845505,"(40.790695, -73.845505)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN
1982065,2023-03-29 11:05:00,MANHATTAN,10011.0,40.735520,-73.993990,"(40.73552, -73.99399)",NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,E-Bike,Sedan


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0e6f0cbc-3bcf-4a76-b278-b525e7427374' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>